In this notebook we will explain the RIPPL workflow based on a study case about the Kumamoto earthquake in 
Japan. 

In the first block we define the study area. There are different options to do so.
1. Create a shapefile (this can be done using ArcGIS or QGIS software)
2. Create a kml file using google earth > https://www.google.com/earth/
3. Create a geojson using > http://geojson.io
4. Create a list of coordinates in lat/lon coordinate pairs. This is what we will do here. 

To show the study area it is possible to run both google earth and geojson within the notebook.

For further background on this event you can read:
https://en.wikipedia.org/wiki/2016_Kumamoto_earthquakes

Following image shows the earthquake strength (credits: USGS 2016):
![Image of Kumamoto earthquake strength](https://upload.wikimedia.org/wikipedia/commons/1/14/Japan_Shakemap_15_April_2016.jpg)

In [12]:
# Import script to embed geolocation.
from IPython.display import HTML
from rippl.orbit_geometry.read_write_shapes import ReadWriteShapes

coordinates = [(130.25, 32.28), (131.35, 32.28), (131.35, 33.24), (130.25, 33.24), (130.25, 32.28)]
study_area = ReadWriteShapes()
study_area(coordinates)

geojson = study_area.shape

# Show and embed geojson.io


The next step in the processing is selection of the right track, as the Sentinel-1 satellite will cover the area on both
ascending and descending tracks. However, to do so, we will have to define the start and end date of our data search, 
because the satellite is not always acquiring data. 
Next search will give a small oversight of the available tracks during our period of interest. In our case we will 
search one week before and one week after the earthquake.

Watch out with extending the search window, this can result in a lot of images. For now, the total number of images
in this software is limited to a 100. Also note that every downloaded file will take approximately 4GB of disk space!

After selection of the right track we can start the actual download of the images. In our case we use track ... in the
descending/ascending direction.

This will download our data automatically to our radar database. Additionally, it will download the precise orbit files.
These files are created within a few weeks after the data acquisition and define the satellite orbit within a few cm
accuracy. These orbits are necessary to accurately define the positions of the radar pixels on the ground later on
in the processing.

In [13]:
from rippl.processing_templates.general_sentinel_1 import GeneralPipelines

start_date = '2016-04-07'
end_date = '2016-04-21'
master_date = '2016-04-08'
track = 156
polarisation = 'VV'
mode = 'IW'
product_type = 'SLC'
stack_name = 'Kumamoto_earthquake'

# Number of processes for parallel processing. Make sure that for every process at least 2GB of RAM is available
no_processes = 4

s1_processing = GeneralPipelines(processes=no_processes)
s1_processing.download_sentinel_data(start_date=start_date, end_date=end_date, track=track,
                                           polarisation=polarisation, shapefile=study_area.shape, data=True)
s1_processing.create_sentinel_stack(start_date=start_date, end_date=end_date, master_date=master_date,
                                          track=track,stack_name=stack_name, polarisation=polarisation,
                                          shapefile=study_area.shape, mode=mode, product_type=product_type)

# Finally load the stack itself. If you want to skip the download step later, run this line before other steps!
s1_processing.read_stack(start_date=start_date, end_date=end_date, stack_name=stack_name)

not possible to connect this time


FileNotFoundError: No images found!

To define the location of the radar pixels on the ground we need the terrain elevation. Although it is possible to 
derive terrain elevation from InSAR data, our used Sentinel-1 dataset is not suitable for this purpose. Therefore, we
download data from an external source to create a digital elevation model (DEM). In our case we use SRTM data. 

However, to find the elevation of the SAR data grid, we have to resample the data to the radar grid first to make it
usable. This is done in the next steps.

In [ ]:
# Some basic settings for DEM creation.
dem_buffer = 0.1        # Buffer around radar image where DEM data is downloaded
dem_rounding = 0.1      # Rounding of DEM size in degrees
dem_type = 'SRTM3'      # DEM type of data we download (SRTM1, SRTM3 and TanDEM-X are supported)

# Define both the coordinate system of the full radar image and imported DEM
s1_processing.create_radar_coordinates()
s1_processing.create_dem_coordinates(dem_type=dem_type)

# Download external DEM
s1_processing.download_external_dem(buffer=dem_buffer, rounding=dem_rounding)

Using the obtained elevation model the exact location of the radar pixels in cartesian (X,Y,Z) and geographic (Lat/Lon)
can be derived. This is only done for the master or reference image. This process is referred to as geocoding.

In [ ]:
# Geocoding of image
s1_processing.geocoding()

The information from the geocoding can directly be used to find the location of the master grid pixels in the slave
grid images. This process is called coregistration. Because the orbits are not exactly the same with every satellite 
overpass but differ hundreds to a few thousand meters every overpass, the grids are slightly shifted with respect to 
each other. These shift are referred to as the spatial baseline of the images. To correctly overlay the master and slave
images the software coregisters and resamples to the master grid.

To do so the following steps are done:
1. Coregistration of slave to master image
2. Deramping the doppler effects due to TOPs mode of Sentinel-1 satellite
3. Resampling of slave image
4. Reramping resampled slave image.

Due to the different orbits of the master and slave image, the phase of the radar signal is also shifted. We do not 
know the exact shift of the two image, but using the geometry of the two images we can estimate the shift of the phase
between different pixels. Often this shift is split in two contributions:
1. The flat earth phase. This phase is the shift in the case the earth was a perfect ellipsoid
2. The topographic phase. This is the phase shift due to the topography on the ground.
In our processing these two corrections are done in one go.

In [ ]:
# Next step applies resampling and phase correction in one step.
# Polarisation
s1_processing.geometric_coregistration_resampling(polarisation=polarisation, output_phase_correction=True)

Before we create an interferogram the different bursts are first mosaicked. This can only be done after resampling as
it is influenced by the phase ramps in TOPs mode of Sentinel-1. 

The independent SAR grids can now be visualized using the amplitude of the resampled data. In our case these are 
written as .tiff files for a georeferenced grid of the region. The data can be visualized using QGIS. The amplitude power
is given in dB. 

In [ ]:
# Resolution of output georeferenced grid
dlat = 0.001
dlon = 0.001

# The actual creation of the calibrated amplitude images
s1_processing.create_ml_coordinates(coor_type='geographic', dlat=dlat, dlon=dlon)
s1_processing.prepare_multilooking_grid(polarisation)
s1_processing.create_calibrated_amplitude_multilooked(polarisation)
s1_processing.create_output_tiffs_amplitude()

After moasicing we can create the interferogram between the different images. This image is also multilooked and 
outputted as a .tiff file. This can also be viewed in QGIS. Because the phase shift between different pixels is often 
larger than two pi radians or a wavelength (56 mm for C-band), this image will show fringes going from -pi to pi and 
starting at -pi again. 
Using the same multilooking grid also a coherence grid is created, which indicates the quality of the obtained phases.

In [ ]:
s1_processing.create_ifg_network(temporal_baseline=30)
s1_processing.create_interferogram_multilooked(polarisation)
s1_processing.create_coherence_multilooked(polarisation)

# Create output geotiffs
s1_processing.create_output_tiffs_coherence_ifg()

# Create lat/lon/incidence angle/DEM for multilooked grid.
s1_processing.create_geometry_mulitlooked()
s1_processing.create_output_tiffs_geometry()


To go to absolute differences the data is therefore unwrapped. The result of this is given in the unwrapped geotiff.

In [ ]:
# To do the unwrapping, we should add a filter step, which is not included in the package at the moment....
# You can do the unwrapping if you change resolution of the image though.
s1_processing.create_unwrapped_images(polarisation)
s1_processing.create_output_tiffs_unwrap()

This finishes the tutorial!